In [16]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer
# import the relevant regressors


# Read File

In [17]:
df = pd.read_csv('StudentsPerformance.csv')

# EDA

In [18]:
print(df.head())

   gender race/ethnicity parental level of education         lunch  \
0  female        group B           bachelor's degree      standard   
1  female        group C                some college      standard   
2  female        group B             master's degree      standard   
3    male        group A          associate's degree  free/reduced   
4    male        group C                some college      standard   

  test preparation course  math score  reading score  writing score  
0                    none          72             72             74  
1               completed          69             90             88  
2                    none          90             95             93  
3                    none          47             57             44  
4                    none          76             78             75  


# Clean the data and separate into training, validation, test tests and target output

Training / validation / test set in 60%/20%/20% ratio

In [ ]:
# Check whether there is any missing data:

df.isna().sum()

gender                         0
race/ethnicity                 0
parental level of education    0
lunch                          0
test preparation course        0
math score                     0
reading score                  0
writing score                  0
dtype: int64

In [ ]:
# Split the dataset

df_all_train, df_test = train_test_split(df, test_size=0.2, random_state=42)
df_train, df_val = train_test_split(df_all_train, test_size=0.25, random_state=42)

ym_train = df_train['math score'].values
ym_val = df_val['math score'].values
ym_all_train = df_all_train['math score'].values
ym_test = df_test['math score'].values

yr_train = df_train['reading score'].values
yr_val = df_val['reading score'].values
yr_all_train = df_all_train['reading score'].values
yr_test = df_test['reading score'].values

yw_train = df_train['writing score'].values
yw_val = df_val['writing score'].values
yw_all_train = df_all_train['writing score'].values
yw_test = df_test['writing score'].values

del df_train['math score']
del df_val['math score']
del df_all_train['math score']
del df_test['math score']

del df_train['reading score']
del df_val['reading score']
del df_all_train['reading score']
del df_test['reading score']

del df_train['writing score']
del df_val['writing score']
del df_all_train['writing score']
del df_test['writing score']

# Use Data Vectorizer
dv = DictVectorizer(sparse=True)

train_dict = df_train.to_dict(orient='records')
all_train_dict = df_all_train.to_dict(orient='records')
val_dict = df_val.to_dict(orient='records')
test_dict = df_test.to_dict(orient='records')

X_train = dv.fit_transform(train_dict)
X_all_train = dv.transform(all_train_dict)
X_val = dv.transform(val_dict)
X_test = dv.transform(test_dict)

# columns:
print(dv.get_feature_names_out())

['gender=female' 'gender=male' 'lunch=free/reduced' 'lunch=standard'
 "parental level of education=associate's degree"
 "parental level of education=bachelor's degree"
 'parental level of education=high school'
 "parental level of education=master's degree"
 'parental level of education=some college'
 'parental level of education=some high school' 'race/ethnicity=group A'
 'race/ethnicity=group B' 'race/ethnicity=group C'
 'race/ethnicity=group D' 'race/ethnicity=group E'
 'test preparation course=completed' 'test preparation course=none']


# Apply Different types of Regressor

- LinearRegressor
- DecisionTreeRegressor

In [ ]:
# Linear Regression

In [ ]:
# Decision Tree Regressor